# Mod 1 - Housing data project

### Importing data

In [65]:
import pandas as pd
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


### Data Cleaning

#### Checking data types for variables

In [66]:
df.info()
#date and sqft_basement columns need to be reformatted

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
id               21597 non-null int64
date             21597 non-null object
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(11), object(2)
memory usage: 3.5+ MB


Converting date column to datetime object

In [67]:
df.date = pd.to_datetime(df['date'], format = '%m/%d/%Y')

Extracting months and years from date column 

In [68]:
df['month'] = df.date.map(lambda x: x.month)
df['year'] = df.date.map(lambda x: x.year)

Converting sqft_basement to integer

In [69]:
pd.DataFrame(df.sqft_basement.value_counts()).head()

,sqft_basement
0.0,12826
?,454
600.0,217
500.0,209
700.0,208


In [70]:
((len(df.loc[df.sqft_basement == '?']) / df.shape[0])) * 100 
#small but not negligible proportion

2.1021438162707784

In [71]:
unknown_base = df.loc[df.sqft_basement == '?', ['sqft_living']].mean()[0]
zero_base = df.loc[df.sqft_basement == '0.0', ['sqft_living']].mean()[0]
non_zero_base = df.loc[(df.sqft_basement != '0.0') & (df.sqft_basement != '?') , ['sqft_living']].mean()[0]
print('?: {}, zero: {}, non-zero: {}'.format(unknown_base, zero_base, non_zero_base))

?: 2052.431718061674, zero: 1929.953609854982, non-zero: 2313.7335577732356


In [72]:
df.loc[df.sqft_basement == '?', ['sqft_basement']] = '0.0'

In [73]:
df.sqft_basement = df.sqft_basement.astype('float64').astype('int64')

#### Checking for missing values

In [74]:
df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
month               0
year                0
dtype: int64

##### dealing with missing waterfront values

In [75]:
pd.DataFrame(df.corr().waterfront).sort_values(by = 'waterfront', ascending = False).head()
#waterfront is most strongly correlated with view

,waterfront
waterfront,1.000000
view,0.406654
price,0.276295
sqft_living,0.110230
sqft_living15,0.088860


In [76]:
null_water = df.loc[df.waterfront.isnull(), ['view']].mean()[0]
zero_water = df.loc[df.waterfront == 0, ['view']].mean()[0]
non_zero_water = df.loc[df.waterfront > 0, ['view']].mean()[0]
print('null: {}, zero: {}, non-zero: {}'.format(null_water, zero_water, non_zero_water))
#mean view value for non-zero waterfront values is considerably larger than for null and zero waterfront values
#replace null waterfront values with zero

null: 0.25654008438818565, zero: 0.20395394079604606, non-zero: 3.786206896551724


In [77]:
df.waterfront.fillna(0, inplace = True)

##### dealing with missing year renovated values

In [79]:
pd.DataFrame(df.yr_renovated.value_counts(normalize=True)).head()
#a large proportion of yr_renovated values are zero
#replace null yr_renovated values with zero

,yr_renovated
0.0,0.958096
2014.0,0.004112
2003.0,0.001746
2013.0,0.001746
2007.0,0.001690


In [80]:
df.yr_renovated.fillna(0, inplace = True)

In [82]:
df['renovated'] = df.yr_renovated.map(lambda x: 0 if x == 0 else 1) 

##### dealing with missing view values

In [83]:
len(df.loc[df.view.isnull()])/df.shape[0] * 100 
#very small proportion of dataset
#remove rows from dataset

0.29170718155299347

In [84]:
df.drop(labels = df.loc[df.view.isnull()].index, axis = 0, inplace = True)

In [85]:
df.isna().sum() #no more nulls

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
month            0
year             0
renovated        0
dtype: int64

#### Checking for outliers

In [86]:
df.describe() 
#appears to be unusually high values for sqft_lot and sqft_lot15

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,month,year,renovated
count,2.153400e+04,2.153400e+04,21534.000000,21534.000000,21534.000000,2.153400e+04,21534.000000,21534.000000,21534.000000,21534.000000,...,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000,21534.000000
mean,4.582351e+09,5.400577e+05,3.373038,2.115712,2079.827854,1.509060e+04,1.494126,0.006734,0.233863,3.409724,...,1971.002275,68.866722,98077.939352,47.560180,-122.213948,1986.299944,12751.079502,6.575555,2014.322560,0.034504
std,2.876779e+09,3.660596e+05,0.926410,0.768602,917.446520,4.138021e+04,0.539806,0.081783,0.765686,0.650654,...,29.376044,364.314552,53.506639,0.138528,0.140735,685.121001,27255.483308,3.113740,0.467467,0.182523
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,...,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000,1.000000,2014.000000,0.000000
25%,2.123212e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,...,1951.000000,0.000000,98033.000000,47.471200,-122.328000,1490.000000,5100.000000,4.000000,2014.000000,0.000000
50%,3.904945e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.617000e+03,1.500000,0.000000,0.000000,3.000000,...,1975.000000,0.000000,98065.000000,47.571900,-122.230000,1840.000000,7620.000000,6.000000,2014.000000,0.000000
75%,7.312175e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068775e+04,2.000000,0.000000,0.000000,4.000000,...,1997.000000,0.000000,98118.000000,47.678100,-122.125000,2360.000000,10083.000000,9.000000,2015.000000,0.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,...,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000,12.000000,2015.000000,1.000000


##### Dealing with sqft_lot outliers

In [96]:
pd.DataFrame(df[['sqft_lot']]).sort_values(by = 'sqft_lot', ascending= False).head(10)
#top value considerably larger than rest

,sqft_lot
1717,1651359
17305,1164794
7640,1074218
7762,1024068
3945,982998
4437,982278
6685,920423
7070,881654
9705,871200
20436,871200


In [97]:
df.drop(labels = df.loc[df.sqft_lot == df.sqft_lot.max()].index, axis = 0, inplace=True)

##### Dealing with sqft_lot15 outliers

In [98]:
pd.DataFrame(df[['sqft_lot15']]).sort_values(by = 'sqft_lot15', ascending= False).head(10)
#top three values are considerably larger than rest

,sqft_lot15
9705,871200
20436,858132
13451,560617
8655,438213
3797,434728
19141,422967
6685,411962
15606,392040
21415,392040
17645,386812


In [101]:
df.drop(labels = df.loc[df.sqft_lot15 == df.sqft_lot15.max()].index, axis = 0, inplace=True)
df.drop(labels = df.loc[df.sqft_lot15 == df.sqft_lot15.max()].index, axis = 0, inplace=True)
df.drop(labels = df.loc[df.sqft_lot15 == df.sqft_lot15.max()].index, axis = 0, inplace=True)

In [ ]:
#import pandas_profiling as pp 

In [ ]:
#df.profile_report(style={'full_width':True})